In [ ]:
#Sorry for this crazy import cell... this will be cleaned up

import psycopg2
import numpy
import nltk
import unicodedata
import pandas as pd
import pprint
import pickle
import re
import os
from nltk.corpus import wordnet
import time
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import nltk.corpus
from nltk.text import TextCollection
import sklearn
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import NuSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from yellowbrick.classifier import ConfusionMatrix

stops = nltk.corpus.stopwords.words('english')
wnl = nltk.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [ ]:
#Input your PostGres credentials to connect

dbname = 'yelp'
username = 'postgres'
host = 'localhost'
password = 'Khoobam1234!'

conn = psycopg2.connect('dbname={} user={} host={} password={}'.format(dbname, username, host, password))
cur = conn.cursor()

In [ ]:
cur = conn.cursor()
cur.execute("""
    SELECT * FROM review LIMIT 2000
""")

cols = ['review_id', 'user_id', 'business_id', 'stars', 'review_date', 'review_text', 'useful', 'funny', 'cool']

review_sample = pd.DataFrame(cur.fetchall(), columns=cols)



In [ ]:
review_sample

In [ ]:
review_sample['review_text']

Contractions:
https://gist.github.com/J3RN/ed7b420a6ea1d5bd6d06

In [ ]:
common_nonneg_contr = ["could've",
"he'd",
"he'd've",
"he'll",
"he's",
"how'd",
"how'll",
"how's",
"i'd",
"i'd've",
"i'll",
"i'm",
"i've",
"it'd",
"it'd've",
"it'll",
"it's",
"let's",
"ma'am",
"might've",
"must've",
"o'clock",
"'ow's'at",
"she'd",
"she'd've",
"she'll",
"she's",
"should've",
"somebody'd",
"somebody'd've",
"somebody'll",
"somebody's",
"someone'd",
"someone'd've",
"someone'll",
"someone's",
"something'd",
"something'd've",
"something'll",
"something's",
"that'll",
"that's",
"there'd",
"there'd've",
"there're",
"there's",
"they'd",
"they'd've",
"they'll",
"they're",
"they've",
"'twas",
"we'd",
"we'd've",
"we'll",
"we're",
"we've",
"what'll",
"what're",
"what's",
"what've",
"when's",
"where'd",
"where's",
"where've",
"who'd",
"who'd've",
"who'll",
"who're",
"who's",
"who've",
"why'll",
"why're",
"why's",
"would've",
"y'all",
"y'all'll",
"y'all'd've",
"you'd",
"you'd've",
"you'll",
"you're",
"you've"]

In [ ]:
neg_stops1 = ['no',
 'nor',
 'not',
 'don',
 "don't",
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'but',
 "don'",
 "ain't"]

words_to_be_added = ['us']
stops1 = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]


In [ ]:
for x in neg_stops1:
    if x in stops1:
        stops1.remove(x)
        
stops1

In [ ]:
new_stops = stops1 + common_nonneg_contr
list(set(new_stops))

# Cell below is preproccessing and tokenezation

In [ ]:
def _remove_stops(tokens):
    stops = nltk.corpus.stopwords.words('english')
    neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
    for x in neg_stops:
        if x in stops:
            stops.remove(x)
            
    tokens_without_stops = [token for token in tokens if token not in stops]
    return tokens_without_stops       
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review2(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = [token for token in text.split() if token not in new_stops]
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    return tokens

def _process_review2(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens







def _process_review(text):
    def _create_stop_words():
        stops = nltk.corpus.stopwords.words('english')
    
        neg_stops = ['no',
         'nor',
         'not',
         'don',
         "don't",
         'ain',
         'aren',
         "aren't",
         'couldn',
         "couldn't",
         'didn',
         "didn't",
         'doesn',
         "doesn't",
         'hadn',
         "hadn't",
         'hasn',
         "hasn't",
         'haven',
         "haven't",
         'isn',
         "isn't",
         'mightn',
         "mightn't",
         'mustn',
         "mustn't",
         'needn',
         "needn't",
         'shan',
         "shan't",
         'shouldn',
         "shouldn't",
         'wasn',
         "wasn't",
         'weren',
         "weren't",
         "won'",
         "won't",
         'wouldn',
         "wouldn't",
         'but',
         "don'",
         "ain't"]

        common_nonneg_contr = ["could've",
        "he'd",
        "he'd've",
        "he'll",
        "he's",
        "how'd",
        "how'll",
        "how's",
        "i'd",
        "i'd've",
        "i'll",
        "i'm",
        "i've",
        "it'd",
        "it'd've",
        "it'll",
        "it's",
        "let's",
        "ma'am",
        "might've",
        "must've",
        "o'clock",
        "'ow's'at",
        "she'd",
        "she'd've",
        "she'll",
        "she's",
        "should've",
        "somebody'd",
        "somebody'd've",
        "somebody'll",
        "somebody's",
        "someone'd",
        "someone'd've",
        "someone'll",
        "someone's",
        "something'd",
        "something'd've",
        "something'll",
        "something's",
        "that'll",
        "that's",
        "there'd",
        "there'd've",
        "there're",
        "there's",
        "they'd",
        "they'd've",
        "they'll",
        "they're",
        "they've",
        "'twas",
        "we'd",
        "we'd've",
        "we'll",
        "we're",
        "we've",
        "what'll",
        "what're",
        "what's",
        "what've",
        "when's",
        "where'd",
        "where's",
        "where've",
        "who'd",
        "who'd've",
        "who'll",
        "who're",
        "who's",
        "who've",
        "why'll",
        "why're",
        "why's",
        "would've",
        "y'all",
        "y'all'll",
        "y'all'd've",
        "you'd",
        "you'd've",
        "you'll",
        "you're",
        "you've"]

        letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
          'u', 'v', 'w', 'x', 'y', 'z']
        
        ranks = ['st', 'nd', 'rd', 'th']
        
        for x in neg_stops:
            if x in stops:
                stops.remove(x)

        new_stops = stops + common_nonneg_contr + letters + ranks + [""] + ['us'] + ['']
        stops = list(set(new_stops))
        return stops

    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].lower()
        tag_dict = {"a": wordnet.ADJ,
                    "n": wordnet.NOUN,
                    "v": wordnet.VERB,
                    "r": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    def _clean_review(text):
        text = text.lower()
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
        tokenizer = nltk.RegexpTokenizer('\w+\'?\w+')
        filtered_tokens = [(re.sub(r"[^A-Za-z\s']", '', token)) for token in tokenizer.tokenize(text)]
        stops = _create_stop_words()
        tokens = [token for token in filtered_tokens if token not in stops]
        tokens = [re.sub("'s", '', token) for token in tokens if re.sub("'s", '', token) != '']
        for i, token in enumerate(tokens):
            tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
        tokens = [token for token in tokens if token != '' and token not in stops]
        return tokens
    
    return _clean_review(text)


In [ ]:
#fyi, tweaked Alice's code here to allow map_on_column to accept another function
def map_on_column(data, function):
    data['review_text'] = data['review_text'].map(lambda x: function(x))
    return data

In [ ]:
map_on_column(review_sample, _process_review)

# Cell below creates a class that will be used in the vectorization code a few cells down. FYI, path in "_load_words" is machine specific. Also, you have to download the GloVe training set from GloVe website.

In [ ]:

class MeanEmbeddingTransformer(TransformerMixin):
    
    def __init__(self):
        self._vocab, self._E = self._load_words()
        
    
    def _load_words(self):
        E = {}
        vocab = []

        with open(r'C:\Users\Mark\YelpProject\poop\glove.6B.200d.txt', encoding="utf8") as file:
            for i, line in enumerate(file):
                l = line.split(' ')
                if l[0].isalpha():
                    v = [float(i) for i in l[1:]]
                    E[l[0]] = numpy.array(v)
                    vocab.append(l[0])
        return numpy.array(vocab), E            

    
    def _get_word(self, v):
        for i, emb in enumerate(self._E):
            if numpy.array_equal(emb, v):
                return self._vocab[i]
        return None
    #in _doc_mean Ithink maybe we should get rid of w.lower.strip
    def _doc_mean(self, doc):
        return numpy.mean(numpy.array([self._E[w.lower().strip()] for w in doc if w.lower().strip() in self._E]), axis=0)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return numpy.array([self._doc_mean(doc) for doc in X])
    
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)
    
    
#retrived from https://www.kaggle.com/nhrade/text-classification-using-word-embeddings

# Cell below is attempting to create a class for gensim doc2vec but it's not working yet and tbh I'm not sure how close I am

In [ ]:
import os
from gensim.corpora import Dictionary
from gensim.matutils import sparse2full

class GensimDoc2Vec(TransformerMixin):
    def __init__(self, path=None):
        self.path = path
        self.id2word = None
        self.load()
    
    def load(self):
        if os.path.exists(self.path):
            self.id2word = Dictionary.load(self.path)
    
    def save(self):
        self.id2word.save(self.path)
    
    def fit(self, documents, labels = None):
        self.id2word = Dictionary(documents)
        self.save()
        return self
    
    def transform(self, documents):
        for document in documents:
            docvec = self.id2word.doc2vec
            yield sparse2full(docvec, len(self.id2word))
    

In [ ]:
def GloVeMeanEmbedding(text):
    met = MeanEmbeddingTransformer()
    GloVeVector = numpy.array(met.fit_transform(map_on_column(text, _clean_review2)))
    return GloVeVector

In [ ]:
def GenDoc2Vec(text, path):
    met = GensimDoc2Vec()
    GenVector = numpy.array(met.transform(map_on_column(text, _clean_review2)))
    return GenVector

# Cell below is vectorization. So far it can do TFIDF on specified number of n-grams, and I think the part with GloVe word embedding is almost functioning, but I haven't started "word2vec" vectorization yet.

In [ ]:
def dummy_fun(doc):
    return doc

#def GloVeDistributedRep(text):
    ##GloVeVector = numpy.append(MeanEmbeddingTransformer().fit_transform(map_on_column(text, _clean_review2))
    #return GloVeVector


def vectorize(text, method, grams):
    if method == 'TFIDF':
        tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,grams))
        TDIFvectors = tfidf.fit_transform(map_on_column(text, _clean_review2))
        print(TDIFvectors)
    if method == 'DistributedRep':
        GloVevectors = GloVeDistributedRep(map_on_column(text, _clean_review2))
        print(GloVevectors)
    if method == 'Word2Vec':
        print('not yet defined')
    else:
        print('choose vectorization method')
        

def vectorize2(text, method, grams):
    if method == 'TFIDF':
        tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,grams))
        TDIFvectors = tfidf.fit_transform(text)
        print(TDIFvectors)
    if method == 'DistributedRep':
        GloVevectors = GloVeDistributedRep(text)
        print(GloVevectors)
    if method == 'Word2Vec':
        print('not yet defined')
    else:
        print('choose vectorization method')
        
        
        
def vectorize3(text, method, grams):
    if method == 'TFIDF':
        tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,grams))
        TDIFvectors = tfidf.fit_transform(text)
        print(TDIFvectors)
    if method == 'DistributedRep':
        GloVevectors = GloVeDistributedRep(text)
        print(GloVevectors)
    if method == 'Word2Vec':
        print('not yet defined')
    else:
        print('choose vectorization method')

In [ ]:


import nltk.corpus
from nltk.text import TextCollection

def TFIDFvectorize(corpus):
    corpus = [_clean_review2(doc) for doc in corpus]
    texts = TextCollection(corpus)
    
    for doc in corpus:
        return {
            term: texts.tf_idf(term, doc)
            for term in doc
        }


# below is first attempt at pipeline. Just as a heads up, some models can't handel negative values and so can't be used with word embedding vectorization. Baysian, for example, can only handle positive vectors and so can be used with TF-IDF but not GloVe word embedding.

In [ ]:


#the Vectorization_method arugment will be one of the classes that we wrote. That argument itself will have argumens within it.
def create_pipeline(documents, estimator, vectorization_method, ngrams, c=10000, reduction=False):
    steps = [
        ('vectorize', vectorization_method) 
    ]
    
    if reduction:
        steps.append((
        'reduction', TruncatedSVD(n_components=c)
        ))
    
    steps.append(('classifier', estimator))
    return Pipeline(steps)



In [ ]:
#this line of code is basically just filler to make sure the pipeline is being used correct. It's functioning output should just be a list of the pipline steps.
create_pipeline(review_sample['review_text'], MultinomialNB, MeanEmbeddingTransformer(), 1)

In [ ]:
#rn the models are just using the default hyperparameters (for exxample, n_neighbors = 5. I'm not sure how to change this. I get an error when I try to change it here.
models = []
for form in (SGDClassifier, KNeighborsClassifier, LogisticRegression, LinearSVC, RandomForestClassifier, SVC, BaggingClassifier):
    models.append(create_pipeline(review_sample['review_text'],form(), MeanEmbeddingTransformer(), 1))

In [ ]:
for model in models:
    model.fit(review_sample['review_text'], review_sample['stars'])

In [ ]:
#really annoyting but rn the printout for this is pretty nasty. 

for model in models:
    scores = []
    model.fit(review_sample['review_text'], review_sample['stars'])
    y_pred = model.predict(review_sample['review_text'])
    score = accuracy_score(review_sample['stars'], y_pred)
    scores.append(score)
    print(model)
    print ("Accuracy of {} is {:03f}".format(model, numpy.mean(scores)))
    print(classification_report(review_sample['stars'], y_pred, labels= [1,2,3,4,5]))


In [ ]:
models3 = []
for form in (MultinomialNB, SGDClassifier, KNeighborsClassifier, LogisticRegression, LinearSVC, RandomForestClassifier):
    models3.append(create_pipeline(review_sample['review_text'],form(), TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun,token_pattern=None, ngram_range=(1,4)), 4))
    

In [ ]:
for model in models3:
    scores = []
    model.fit(review_sample['review_text'], review_sample['stars'])
    y_pred = model.predict(review_sample['review_text'])
    score = accuracy_score(review_sample['stars'], y_pred)
    scores.append(score)
    print ("Accuracy of {} is {:03f}".format(model, numpy.mean(scores)))
    y_pred = model.predict(review_sample['review_text'])
    print(classification_report(review_sample['stars'], y_pred, labels= [1,2,3,4,5]))
    #cm = ConfusionMatrix(model, classes=[1,2,3,4,5])
    #cm.fit(review_sample['review_text'], review_sample['review_text'])
    #cm.score(review_sample['review_text'], review_sample['review_text'])
    #print(cm.poof())

# Below is a visualization of our data. I was hoping that some visual clusters would pop out but no luck. If you get a  n_components must be < n_features; got 50 >= 50 error then set n_components less than 50 in TSNE visualizer.

In [ ]:
from yellowbrick.text import TSNEVisualizer
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizor = MeanEmbeddingTransformer()
vectorizor1 = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,2))

docs = vectorizor.fit_transform(review_sample['review_text'])
labels = review_sample['stars']


tsne = TSNEVisualizer(metric= "euclidean", size=(1080, 720))
tsne.fit(docs, labels)

print(tsne.poof())

In [ ]:
from yellowbrick.text import TSNEVisualizer
from sklearn.feature_extraction.text import TfidfVectorizer

grams = [1,2,3,5]

for n in grams:
    vectorizor = MeanEmbeddingTransformer()
    vectorizor1 = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,n))
    docs = vectorizor1.fit_transform(review_sample['review_text'])
    labels = review_sample['stars']
    tsne = TSNEVisualizer(size=(1080, 720))
    tsne.fit(docs, labels)
    print(tsne.poof())
